In [ ]:
!pip install numpy==1.26.4
!pip install datasets==3.2.0
!pip install transformers==4.11.3
!pip install huggingface_hub==0.27.1
!pip install evaluate==0.4.3
!pip install torchaudio
!pip install librosa
!pip install jiwer

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
^C
Traceback (most recent call last):
  File "/home/vshilonosov/miniconda3/bin/pip", line 11, in <module>
    sys.exit(main())
             ^^^^^^
  File "/home/vshilonosov/miniconda3/lib/python3.12/site-packages/pip/_internal/cli/main.py", line 79, in main
    return command.main(cmd_args)
           ^^^^^^^^^^^^^^^^^^^^^^
  File "/home/vshilonosov/miniconda3/lib/python3.12/site-packages/pip/_internal/cli/base_command.py", line 101, in main
    return self._main(args)
           ^^^^^^^^^^^^^^^^
  File "/home/vshilonosov/miniconda3/lib/python3.12/site-packages/pip/_internal/cli/base_command.py", line 236, in _main
    self.handle_pip_version_check(options)
  File "/home/vshilonosov/miniconda3/lib/python3.12/site-packages/pip/_internal/cli/req_command.py", line 188, in handle_pip_version_check
    pip_self_version_check(session, options)
  File "/home/vshilonosov/miniconda3/lib/python3.12/site-packages/pip/_intern

In [192]:
from huggingface_hub import notebook_login
from datasets import load_dataset, Audio, ClassLabel
import evaluate
from dotenv import load_dotenv
import pandas as pd
from IPython.display import display, HTML
import IPython.display as ipd
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2ForCTC, Wav2Vec2FeatureExtractor, Wav2Vec2Processor, Trainer
import numpy as np
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
import os
import json
import random
import re

In [172]:
notebook_login()

In [78]:
common_voice_train = load_dataset("mozilla-foundation/common_voice_17_0", "ru", split="train+validation")
common_voice_test = load_dataset("mozilla-foundation/common_voice_17_0", "ru", split="test")

ru_train_0.tar:   0%|          | 0.00/976M [00:00<?, ?B/s]

ru_dev_0.tar:   0%|          | 0.00/392M [00:00<?, ?B/s]

ru_test_0.tar:   0%|          | 0.00/397M [00:00<?, ?B/s]

ru_other_0.tar:   0%|          | 0.00/597M [00:00<?, ?B/s]

ru_invalidated_0.tar:   0%|          | 0.00/387M [00:00<?, ?B/s]

...


ding metadata...: 0it [00:00, ?it/s]
Reading metadata...: 17456it [00:00, 129480.70it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 10018it [00:00, 139775.25it/s]


Generating validated split: 0 examples [00:00, ? examples/s]


ding metadata...: 0it [00:00, ?it/s]
ding metadata...: 15958it [00:00, 159566.08it/s]
ding metadata...: 31915it [00:00, 130767.30it/s]
ding metadata...: 45315it [00:00, 127289.65it/s]
ding metadata...: 58198it [00:00, 123757.51it/s]
ding metadata...: 72074it [00:00, 128737.60it/s]
ding metadata...: 85042it [00:00, 125619.91it/s]
ding metadata...: 97666it [00:00, 125357.15it/s]
ding metadata...: 110242it [00:00, 124215.86it/s]
ding metadata...: 123981it [00:00, 128187.79it/s]
ding metadata...: 136832it [00:01, 126097.04it/s]
Reading metadata...: 163387it [00:01, 128540.59it/s]


In [79]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [80]:
show_random_elements(common_voice_train.remove_columns(["path","audio"]))

,sentence,variant
0,В Йемене выборы президента Хади стали важной вехой в политическом переходном процессе.,
1,Главный вопрос проистекает из коренных причин подобных подходов.,
2,Я говорил это Аркадию на другой день его приезда и повторяю теперь вам.,
3,Отслеживать состояние ресурсов каждого участника системы,
4,И Япония готова вносить конструктивный вклад в обсуждение этого вопроса.,
5,Мы выслушали этих женщин и были просто поражены.,
6,Мы заслушали последнего оратора из списка ораторов на этом заседании.,
7,Пришло время пойти на риск.,
8,— Проснись и крикни.,
9,"Люди увидали в миллион раз больше звезд, или солнц, чем видели раньше.",


In [131]:
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\'\’\…\(\)\—\−\„\‑\–\«\»]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_remove_regex, '', batch["sentence"]).lower()
    return batch

<>:1: SyntaxWarning: invalid escape sequence '\,'
<>:1: SyntaxWarning: invalid escape sequence '\,'
/tmp/ipykernel_4144880/3899105336.py:1: SyntaxWarning: invalid escape sequence '\,'
  chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\'\’\…\(\)\—\−\„\‑\–\«\»]'


In [132]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

Map:   0%|          | 0/36580 [00:00<?, ? examples/s]

Map:   0%|          | 0/10203 [00:00<?, ? examples/s]

In [133]:
show_random_elements(common_voice_train.remove_columns(["path","audio"]))

,sentence,variant
0,почти,
1,как не задуматься отвечал я ему,
2,финансирование остается для нас большой проблемой,
3,а сейчас слово имеет представитель канады,
4,планируется его продолжить,
5,это должно быть прямо закреплено,
6,поэтому мы на самом деле нуждаемся не в поддержании мира а в его укреплении,
7,она два раза прокуковала обычным криком а потом захрипела заторопилась и запуталась,
8,поэтому прямые иностранные инвестиции должны дополняться международным финансовым и техническим сотрудничеством в целях развития,
9,что тебе делать,


In [136]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [137]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

Map:   0%|          | 0/36580 [00:00<?, ? examples/s]

Map:   0%|          | 0/10203 [00:00<?, ? examples/s]

In [138]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [139]:
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict

{' ': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'k': 9,
 'l': 10,
 'm': 11,
 'n': 12,
 'o': 13,
 'p': 14,
 'r': 15,
 's': 16,
 't': 17,
 'x': 18,
 'z': 19,
 'а': 20,
 'б': 21,
 'в': 22,
 'г': 23,
 'д': 24,
 'е': 25,
 'ж': 26,
 'з': 27,
 'и': 28,
 'й': 29,
 'к': 30,
 'л': 31,
 'м': 32,
 'н': 33,
 'о': 34,
 'п': 35,
 'р': 36,
 'с': 37,
 'т': 38,
 'у': 39,
 'ф': 40,
 'х': 41,
 'ц': 42,
 'ч': 43,
 'ш': 44,
 'щ': 45,
 'ъ': 46,
 'ы': 47,
 'ь': 48,
 'э': 49,
 'ю': 50,
 'я': 51,
 'ё': 52}

In [140]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [141]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

55

In [142]:
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [174]:
repo_name = "wav2vec2-large-xls-r-300m-ru"

In [175]:
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [176]:
tokenizer.push_to_hub(repo_name)

CommitInfo(commit_url='https://huggingface.co/NLPVladimir/wav2vec2-large-xls-r-300m-ru/commit/1685e8a089591d212a5748c8c776719b8a1ad9e1', commit_message='Upload tokenizer', commit_description='', oid='1685e8a089591d212a5748c8c776719b8a1ad9e1', pr_url=None, repo_url=RepoUrl('https://huggingface.co/NLPVladimir/wav2vec2-large-xls-r-300m-ru', endpoint='https://huggingface.co', repo_type='model', repo_id='NLPVladimir/wav2vec2-large-xls-r-300m-ru'), pr_revision=None, pr_num=None)

In [177]:
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [178]:
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [180]:
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))

In [181]:
rand_int = random.randint(0, len(common_voice_train)-1)

print(common_voice_train[rand_int]["sentence"])
ipd.Audio(data=common_voice_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)

KeyError: 'sentence'

In [182]:
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["sentence"])
print("Input array shape:", common_voice_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", common_voice_train[rand_int]["audio"]["sampling_rate"])

KeyError: 'sentence'

In [183]:
def prepare_dataset(batch):
    audio = batch["audio"]

    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch


In [151]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names)

Map:   0%|          | 0/36580 [00:00<?, ? examples/s]

2025-01-30 14:22:47.648272: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738236167.786979 4144880 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738236167.827865 4144880 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-30 14:22:48.161571: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/vshilonosov/miniconda3/lib/python3.12/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarnin

Map:   0%|          | 0/10203 [00:00<?, ? examples/s]

In [184]:
@dataclass
class DataCollatorCTCWithPadding:
    
    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:

        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch


In [185]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [186]:
wer_metric = evaluate.load("wer")

In [216]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [217]:
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m", 
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [218]:
model.freeze_feature_encoder()

In [224]:
training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=2,
  gradient_accumulation_steps=2,
  eval_strategy="steps",
  num_train_epochs=1,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=2,
  eval_steps=2,
  logging_steps=2,
  learning_rate=3e-4,
  warmup_steps=2,
  save_total_limit=2,
  push_to_hub=True,
)

In [225]:
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    processing_class=processor.feature_extractor,
)


Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
trainer.train()